The following is Dijkstra's algorithm. 

In [1]:
import math
# CHATGPT: Give Python code to find the distance to all nodes from a starting node. 
# The input is a dictionary with nodes as keys and list of connected nodes as values. 
import heapq
def dijkstra(graph, start):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    priority_queue = [(0, start)]
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_distance > distances[current_node]: continue
        for neighbor in graph[current_node]:
            distance = current_distance + 1
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
    return distances

Set to test mode:

In [3]:
small = False

Load data and build graph. 

In [4]:
with open('input7' + ('A' if small else 'B') + '.txt', 'r') as i:
    data = list(list(line.strip()) for line in i.readlines())
    M = 5000 if small else 26501365 # is het aantal stappen

h = len(data)
w = len(data[0])
assert(w == h)

graph = {}

enc = lambda y, x: x * h + y
dec = lambda v: [v % h, v // h]
ends = []
for y in range(h):
    for x in range(w):
        l = []
        if data[y][x] in '.S':
            if data[y][x] == 'S': start = enc(y, x)
            if y >     0 and data[y - 1][x] in '.S': l += [enc(y - 1, x)]
            if y < h - 1 and data[y + 1][x] in '.S': l += [enc(y + 1, x)]
            if x >     0 and data[y][x - 1] in '.S': l += [enc(y, x - 1)]
            if x < w - 1 and data[y][x + 1] in '.S': l += [enc(y, x + 1)]
            graph[enc(y, x)] = l

nodes = set(graph.keys())

In [5]:
hoekjes = [
    enc(0, 0), # lower-right
    enc(h - 1, 0), # upper right
    enc(h - 1, w - 1), # upper left
    enc(0, w - 1), # lower left
]
kantjes = [
    enc(0, w // 2), 
    enc(h - 1, w // 2), 
    enc(h // 2, 0), 
    enc(h // 2, w - 1)
]
def omgekeerd(e):
    x, y = dec(e)
    return enc(w - 1 - x, w - 1 - y)
def paar_afstand(e):
    x, y = dec(e)
    x2, y2 = dec(omgekeerd(e))
    return int(x == x2) + int(y == y2)

In [6]:
count = lambda start, max_step=math.inf: len([key 
                                              for key, val in dijkstra(graph, start).items()
                                              if val <= max_step and 
                                                 ((max_step if max_step != math.inf else 0) - val) % 2 == 0])

In [7]:
aantal_per_blok_totaal = len(list(1 for row in data for item in row if item in 'S.'))
print((aantal_per_blok_totaal, count(start) + count(start, 100001)))
print([dec(key) for key, val in dijkstra(graph, start).items() if val == math.inf and data[dec(key)[0]][dec(key)[1]] != "#"])

(14677, 14669)
[[2, 29], [3, 88], [24, 63], [31, 89], [32, 88], [32, 89], [33, 20], [115, 90]]


In [8]:
odd = lambda x: ((x - 1) // 2) * 2 + 1
even = lambda x: (x // 2) * 2
sum_odd_numbers = lambda max: (odd(max) + 1) ** 2 // 4
sum_even_numbers = lambda max: even(max) * (even(max) + 2) // 4
sum_numbers = lambda max, sign: sum_odd_numbers(max) if sign == 1 else sum_even_numbers(max)
assert(sum_even_numbers(11) == 2 + 4 + 6 + 8 + 10)
assert(sum_odd_numbers(11) == 1 + 3 + 5 + 7 + 9 + 11)

In [15]:
center = count(start, M)
def chunk(start0, aantallen):
    L = max(filter(lambda x: x != float('inf'), dijkstra(graph, start0).values())) # is 2 * (w - 1) for hoekjes
    K = dijkstra(graph, start)[omgekeerd(start0)] + paar_afstand(start0) # 2 als hoekje, 1 als kantje, moet 66 bij kantjes zijn
    k = math.floor((M - K - L) / w)
    #print(L)
    #print(K)
    #print((M - K - L) / w)
    print(k)
    print(((M - K) - k * w) / w)

    start_parity = K % 2
    volledig_gevuld = (aantallen(k)[0] * count(start0, L + start_parity) + 
                    aantallen(k)[1] * count(start0, L + (1 - start_parity))) # onafhankelijk van hoekje
    print(volledig_gevuld)
    extra1 = aantallen(k)[2] * count(start0, M - K - (k + 1) * w)
    extra2 = aantallen(k)[3] * count(start0, M - K - (k + 2) * w)
    return [volledig_gevuld, extra1, extra2]


c = sum([sum(chunk(hoekje, lambda k: [sum_numbers(k, 0), sum_numbers(k, 1), (k + 1), (k + 2)])) for hoekje in hoekjes])
a = sum([sum(chunk(kantje, lambda k: [k - (k // 2), k // 2, 1, 1])) for kantje in kantjes]) # not assuming K is odd

totaal = center + c + a


202297
2.50381679389313
150079558378723
202297
2.50381679389313
150079558378723
202297
2.50381679389313
150079558378723
202297
2.50381679389313
150079558378723
202298
1.9923664122137406
1483754681
202298
1.9923664122137406
1483754681
202298
1.9923664122137406
1483754681
202298
1.9923664122137406
1483754681


In [16]:
totaal

600330215658769

In [ ]:
# 600336166486629
# 600330215658769